<a href="https://colab.research.google.com/github/rgmerski/RegisterApp/blob/master/Notebooks/Data_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Początkowe rozmyślenia:


*   objekty - z granicami czy bez?
*   kilka obiektów - rozdzielone? 
*   kolory

* bardziej skomplikowane figury (obiekty)



# 1. Przygotowanie środowiska

Sprawdzanie podłączenia GPU

In [ ]:
%tensorflow_version 2.x
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
tf.__version__

'2.4.1'

# 2. Data Augmentation

Data Augumentation - wykonać raz po dodaniu podstawowych danych.
7 kolorów trójkątów, kwadratów oraz kół.
Planowane zmiany w zdjęciach - datagen. 
Odnośnie klas z wieloma/x objektami - nie wiem, czy objekty powinny się na siebie nakładać.

[Dokumentacja ImageDataGenerator - Keras](https://keras.io/api/preprocessing/image/#imagedatagenerator-class)

## 2.1. Wczytanie podstawowego datasetu (bez DA) w celu dodania zedytowanych obrazów

[Dokumentacja image_dataset_from_directory](https://keras.io/api/preprocessing/image/#imagedatasetfromdirectory-function)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/MyDrive/dataset', 
    labels='inferred',
    label_mode="categorical",
    batch_size=16
    )

Found 4718 files belonging to 6 classes.


In [ ]:
classes = [1, 2, 3, 4, 5, 6]

usuwanie zaugmentowanych:

In [ ]:
import os, glob
for filename in glob.glob("/content/drive/MyDrive/dataset/1/aug*"):
    os.remove(filename) 

##2.2. Usuwanie zaugumentowanych (wszystkie klasy)

In [ ]:
import os, glob

for cls in classes:
  for filename in glob.glob("/content/drive/MyDrive/dataset/" + str(cls) + "/aug*"):
      os.remove(filename) 

ilość zaugmentowanych:

In [ ]:
import os, glob
n=0
for cls in classes:
  for filename in glob.glob("/content/drive/MyDrive/dataset/" + str(cls) + "/aug*"):
     n+=1
print(n)

3350


## 2.3. Pobieranie ścieżek obrazów w celu ich edycji

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
import os
import numpy as np

In [ ]:
# tworzenie list ścieżek do danych
# paths - lista list, np paths[0] -> path_1 nizej

paths = []

for cls in classes:

  path_temp = []
  directory = r'/content/drive/MyDrive/dataset/' + str(cls)

  # skanuj w poszukiwaniu obrazów, dodawaj ich ścieżkę do path_temp
  for entry in os.scandir(directory):
    if (entry.path.endswith(".png") or entry.path.endswith(".jpg")) and entry.is_file():
        #print(entry.path)
        path_temp.append(entry.path)
  paths.append(path_temp[:])

#paths[x] - ścieżki do plików, gdzie x-iteracja po klasach (od zera), przykładowo:
#paths[0] - pierwsza klasa z listy classes, tutaj - 1
#paths[x] - ścieżki do classes[x]


**NIE MOŻNA URUCHOMIĆ PONIŻSZEGO KODU!!! Zapisanie wszystkich aktualnych obrazków w formie tabeli wymaga większych zasobów RAM**
Chciałem zautomatyzować wszystko, by można było na raz wszystko uczynić, lecz niestety. Sam proces augmentacji wymaga podzielenia na kilka części.

In [ ]:
#NIE MOŻNA TEGO URUCHOMIĆ, ZA MAŁO RAM!!!!!!!!
#images = []

# iteracja po listach - path_ kolejne listy ze ścieżkami, p - ścieżki
#for path_ in paths:
  
#  images_temp = []
#------------------------------------------  
  #for p in path_:
   # image = load_img(p)
   # image = img_to_array(image)
   # image = np.expand_dims(image, axis = 0)
   # images_temp.append(image)

 # images.append(images_temp[:])
#-------------------------------------------

In [ ]:
'''
image_array(images, n)
PARAMS:
images - lista, do której funkcja wpisuje obrazy w formie tablic z wcześniej uzyskanych ścieżek plików (paths)
n - która lista ze ścieżkami do obrazów, czyli nr klasy pomniejszony o 1 (poprawne wyłuskanie informacji z listy paths)
'''
def image_array(images, n):
  for p in paths[n]:
    #print(p)
    image = load_img(p)
    image = img_to_array(image)
    image = np.expand_dims(image, axis = 0)
    images.append(image)

## 2.4. Augmentacja

In [ ]:
datagen_1 = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.1,
    rotation_range=90,
    width_shift_range=0.05,
    height_shift_range=0.05,
    horizontal_flip=True,
    #vertical_flip=True
    )


In [ ]:
datagen_more = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.15,
    zoom_range=0.05,
    rotation_range=90,
    #width_shift_range=0.05,
    #height_shift_range=0.05,
    #horizontal_flip=True,
    #vertical_flip=True
    )


In [ ]:
'''
DA(augs, images_, folder)
Dokonuje augmentacji na podstawie generatora datagen (pomniejsze zmiany aby nie zniszczyć zbytnio danych dla datagen_more) i od razu je zapisuje, możliwość czyszczenia RAM
PARAMS:
augs - ilość augmentacji na zdjęcie
images - tablica zawierająca informacje o zdjęciach
folder - zapis (klasa zdjęć)
mozna dodać różne generatory
'''
def DA(augs, images_, folder, datagen=datagen_more):
  for image in images_:
    temp = '/content/drive/MyDrive/dataset/' + str(folder)
    imageGen = datagen.flow(image, batch_size=1, save_to_dir=temp, save_prefix='aug', save_format="png")
    total = 0
    for img in imageGen:
      total += 1
      if total == augs:
        break
  images_ = []

ABY ZMIEŚCIĆ SIĘ W MOŻLIWEJ POJEMNOŚCI PAMĘCI RAM

In [ ]:
images_2 = []
image_array(images_2, 1)

In [ ]:
DA(2, images_2, 2)
images_2 = []

In [ ]:
images_3 = []
image_array(images_3, 2)

DA(2, images_3, 3)
images_3 = []


images_4 = []
image_array(images_4, 3)

DA(2, images_4, 4)
images_4 = []


images_5 = []
image_array(images_5, 4)

DA(2, images_5, 5)
images_5 = []


images_6 = []
image_array(images_6, 5)

DA(2, images_6, 6)
images_6 = []



Budowa nazwy plików: 
* aug_ - zaugmentowane dane
* aug_x_ - dane zaugmentowane z ran_x
* aug_ - dane zaugmentowane z bazowych figur
* ran_x - losowe figury wygenerowane przy pomocy kodu


Kod niżej - testowy! Sprawdzałem zgodność plików, gdyż kilka obrazów w klasie pierwszej stworzyłem ręcznie.

In [ ]:
augs = 5
images_1 = []
image_array(images_1, 0)
# w celu rozpoznania początkowego - iterator
real = 0
real_it = 0

total = 0
for image in images_1:

  if real_it > 20:
    temp = 'aug_' + str(real)
    imageGen = datagen_1.flow(image, batch_size=1, save_to_dir='/content/drive/MyDrive/dataset/1', save_prefix=temp, save_format="png")
    real += 1
  else:
    imageGen = datagen_1.flow(image, batch_size=1, save_to_dir='/content/drive/MyDrive/dataset/1', save_prefix='aug', save_format="png")
  
  real_it += 1
  total = 0
  
  for img in imageGen:
    total += 1
    if total == augs:
      break
images_1 = []